In [1]:
import os
from PIL import Image

# Open test.txt and read the lines
with open(os.path.join(os.pardir, '2_test.txt'), 'r') as file:
  test_items = file.read().splitlines()

images = []

for item in test_items:
  image_path = os.path.join(os.pardir, 'ArtDL', 'JPEGImages', f"{item}.jpg")
  try:
    image = Image.open(image_path)
    images.append(image)
  except Exception as e:
    print(f"Error loading image {image_path}: {e}")

print(f"Loaded {len(images)} images")

Loaded 1864 images


Test SigLIP with these models:

* google/siglip-base-patch16-512
* google/siglip-large-patch16-384

In [2]:
from transformers import AutoProcessor, AutoModel
from tqdm import tqdm
import pandas as pd
import torch

models = ["siglip-base-patch16-512", "siglip-large-patch16-384", "siglip-so400m-patch14-384"]
model_name = models[2]
print(f"Using model {model_name}")

model = AutoModel.from_pretrained("google/"+model_name)
processor = AutoProcessor.from_pretrained("google/"+model_name)

# Load classes
with open(os.path.join('classes_desc.txt'), 'r') as f:
  classes = [tuple(line.strip().split(',')) for line in f]

all_probs = []
for idx, image in enumerate(tqdm(images)):
  inputs = processor(text=[f"This is a photo of {cls[1]}" for cls in classes], images=image, padding="max_length", return_tensors="pt")

  with torch.no_grad():
    outputs = model(**inputs)

  logits_per_image = outputs.logits_per_image
  probs = torch.sigmoid(logits_per_image)
  all_probs.append(probs)
  
all_probs = torch.cat(all_probs, dim=0)
print(f"Probabilities shape {all_probs.shape}")

# Convert all_probs to a DataFrame and store it as a CSV file
all_probs_df = pd.DataFrame(all_probs.numpy())
os.makedirs(os.path.join(os.curdir, 'evaluations', model_name), exist_ok=True)
torch.save(all_probs, os.path.join(os.curdir, 'evaluations', model_name, 'probs.pt'))


/Users/gspinaci/projects/llm-art/LLM-test/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using model siglip-so400m-patch14-384


100%|██████████| 1864/1864 [40:24<00:00,  1.30s/it]

Probabilities shape torch.Size([1864, 10])
